In [10]:
"""
This script can be used as skelton code to read the challenge train and test
csvs, to train a trivial model, and write data to the submission file.
"""
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import seaborn as sns
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict

In [2]:
from sklearn.metrics import accuracy_score

## Read csvs
train_df = pd.read_csv('train.csv', index_col=0)
test_df = pd.read_csv('test.csv', index_col=0)

In [3]:
## Handle missing values
train_df.fillna('NA', inplace=True)
test_df.fillna('NA', inplace=True)

In [4]:
## Filtering column "mail_type"
train_x = train_df[['org','tld','mail_type']]
train_y = train_df[['label']]

test_x = test_df[['org','tld','mail_type']]

#sns.distplot(int(train_df['mail_type']), kde=False)


In [5]:
## PCA
data_train = np.array(train_df.iloc[:, [3,4,6,7,8,9,10,11]])
data_test = np.array(test_df.iloc[:, [3,4,6,7,8,9,10,11]])
print(len(data_test))
data = np.vstack((data_train, data_test))#0:10744是test

M = np.mean(data, 0) # compute the mean
Var = np.var(data,0)
# C = data-M
C = (data - M)*1/Var
W = np.dot(C.T, C) # compute covariance matrix
eigval, eigvec = np.linalg.eig(W) # compute eigenvalues and eigenvectors of covariance matrix
idx = eigval.argsort()[::-1] # Sort eigenvalues
eigvec = eigvec[:,idx] # Sort eigenvectors according to eigenvalues
print(eigval/sum(eigval))
newData2 = np.dot(C,np.real(eigvec[:,:2])) # Project the data to the new space (2-D)
newData3 = np.dot(C,np.real(eigvec[:,:5])) # Project the data to the new space (3-D)

newData2_train = newData2[0:25066]
newData2_test = newData2[25066:35811]

newData3_train = newData3[0:25066]
newData3_test = newData3[25066:35811]


print(newData3_train)
len(newData2_train)
print(len(newData3_test))
# newData3 = np.dot(C,np.real(eigvec[:,:3])) # Project the data to the new space (3-D)




10745
[9.83844148e-01 1.14345575e-02 4.60218639e-03 1.08486910e-04
 9.52668873e-06 9.32368512e-07 1.62504060e-07 6.52661555e-16]
[[ 1.00430849e+00  9.01661897e+00  2.54263278e+00 -5.78249599e-03
   1.01436879e-01]
 [ 1.00329378e+00 -1.28032529e+00  1.47954552e+00  1.72461679e-02
  -6.50377730e-02]
 [ 9.97216738e-01 -8.43302024e-01 -2.75785387e+00 -1.68705528e-02
  -4.87539980e-02]
 ...
 [ 1.00329573e+00 -1.28026680e+00  1.47969462e+00  1.78300917e-02
  -3.88827259e-02]
 [ 1.00329310e+00 -1.28036221e+00  1.47949041e+00  1.69896449e-02
  -7.46349278e-02]
 [ 9.97234820e-01 -8.42722549e-01 -2.75528602e+00  8.70769210e-02
   1.27733146e-01]]
10745


In [6]:
## Do one hot encoding of categorical feature
feat_enc = OneHotEncoder()
feat_enc.fit(np.vstack((train_x, test_x)))
train_x_featurized = feat_enc.transform(train_x)
test_x_featurized = feat_enc.transform(test_x)

train_type_array = train_x_featurized.A
test_type_array = test_x_featurized.A

data_type = np.vstack((train_type_array, test_type_array))

M0 = np.mean(data_type, 0) # compute the mean
Var0 = np.var(data_type,0)
C0 = (data_type - M0)*1/Var0

train_type_array = C0[0:25066]
test_type_array = C0[25066:35811]


newData3_train_fin = np.hstack((newData3_train, train_type_array))
newData3_test_fin = np.hstack((newData3_test, test_type_array))
print(newData3_test_fin)
newData3_train_fin
type(test_type_array)
#type(train_type_array = train_x_featurized.A)

[[ 0.99720253 -0.84360155 -2.76244782 ... -1.02124565 -1.14952011
  -1.03207678]
 [ 1.00329327 -1.28033182  1.47950269 ... -1.02124565 -1.14952011
  -1.03207678]
 [ 0.99722231 -0.84320977 -2.75626607 ... -1.02124565 -1.14952011
  -1.03207678]
 ...
 [ 1.00330611 -1.27992041  1.48049883 ... -1.02124565  7.68806355
  -1.03207678]
 [ 1.00329573 -1.28026791  1.47969443 ... -1.02124565 -1.14952011
  -1.03207678]
 [ 0.99722169 -0.84320448 -2.75630833 ... -1.02124565  7.68806355
  -1.03207678]]


numpy.ndarray

In [32]:
## Train a simple KNN classifier using featurized data
neigh = KNeighborsClassifier(n_neighbors=150)
neigh.fit(newData3_train, train_y)
pred_y = neigh.predict(newData3_test)

pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("knn_submission.csv", index=True, index_label='Id')


/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
##Train a SVM classifier
clf = LinearSVC(random_state=0, tol=1e-5)
clf.fit(newData2_train, train_y)
pred_y = clf.predict(newData2_test)

pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("svm_submission.csv", index=True, index_label='Id')


/usr/local/lib/python3.7/site-packages/sklearn/utils/validation.py:744: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/site-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [12]:
##Train a NNC classifier
clf = MLPClassifier(batch_size=80)
clf.fit(newData3_train_fin, train_y)

pred_y = clf.predict(newData3_test_fin)

pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("nn_submission.csv", index=True, index_label='Id')


/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
clf_rdf = RandomForestClassifier()
clf_rdf.fit(newData3_train_fin,train_y)

pred_y = clf_rdf.predict(newData3_test_fin)

pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("rdf_submission.csv", index=True, index_label='Id')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


In [ ]:
clf_GB = GradientBoostingClassifier()
clf_GB.fit(newData3_train_fin,train_y)

pred_y = clf_GB.predict(newData3_test_fin)

pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("GB_submission.csv", index=True, index_label='Id')


In [10]:
batch_size = [10, 20, 40, 60, 80, 100, 120, 140] 
epochs = [10, 50, 100, 150, 200] 
param_grid = dict(batch_size=batch_size) 

grid = GridSearchCV(estimator=clf, param_grid=param_grid, n_jobs=-1) 

grid_result = grid.fit(newData3_train_fin, train_y) 

# summarize results 

print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_)) 

for params, mean_score, scores in grid_result.grid_scores_: 

    print("%f (%f) with: %r" % (scores.mean(), scores.std(), params))

pred_y = grid_result.predict(newData3_test_fin)

pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("nn_GS_submission.csv", index=True, index_label='Id')

/usr/local/lib/python3.7/site-packages/sklearn/neural_network/_multilayer_perceptron.py:934: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Best: 0.901620 using {'batch_size': 80}


AttributeError: 'GridSearchCV' object has no attribute 'grid_scores_'

In [20]:
## Save results to submission file
pred_df = pd.DataFrame(pred_y, columns=['label'])
pred_df.to_csv("knn_sample_submission.csv", index=True, index_label='Id')
